In [36]:
import pandas as pd
import panel as pn
import plotly.express as px
from bokeh.resources import INLINE

# Cargar datos desde el archivo Excel
df = pd.read_excel('/home/enrique/Descargas/FGL/Relevamiento_escuelas_de_golf.xlsx')

# Preprocesamiento de datos
# Combinar columnas de latitud/longitud (maneja posibles variantes de nombre con o sin acento)
if '_Ubicación_latitude' in df.columns:
    df['latitude'] = df['_Ubicación_latitude'].fillna(df.get('_Ubicacion_latitude'))
    df['longitude'] = df['_Ubicación_longitude'].fillna(df.get('_Ubicacion_longitude'))
else:
    # Si solo existe una de las variantes
    df['latitude'] = df.get('_Ubicacion_latitude', df.get('_Ubicación_latitude'))
    df['longitude'] = df.get('_Ubicacion_longitude', df.get('_Ubicación_longitude'))

# Crear columna de total de alumnos sumando juniors y menores
df['Total de alumnos'] = df['Cantidad de Juniors (menores de 13 años) en la escuela'] + df['Cantidad de Menores (13 a 18 años) en la escuela']

# Reemplazar NaN por 0 en columnas de opciones múltiples (checkboxes) para poder contar correctamente
multi_cols = [col for col in df.columns if col.startswith('Cuales?/') or col.startswith('¿Qué ')]
for col in multi_cols:
    df[col] = df[col].fillna(0).astype(int)

# Inicializar Panel con extensiones de Plotly y Tabulator
pn.extension('plotly', 'tabulator')

# Definir widgets de filtro (menú lateral)
club_filter = pn.widgets.MultiChoice(name='Filtrar por club', options=list(df['Nombre del club'].unique()), value=[])
has_range_filter = pn.widgets.Checkbox(name='Solo clubes con Driving Range', value=False)
has_materials_filter = pn.widgets.Checkbox(name='Solo clubes con materiales didácticos', value=False)

# Función de utilidad para aplicar filtros a los datos
def filtrar_datos(clubs, has_range, has_materials):
    df_filtered = df.copy()
    if clubs:
        df_filtered = df_filtered[df_filtered['Nombre del club'].isin(clubs)]
    if has_range:
        df_filtered = df_filtered[df_filtered['Cuenta con una zona de práctica (driving range)?'] == 'Si']
    if has_materials:
        df_filtered = df_filtered[df_filtered['Cuenta con materiales didácticos en la escuela?'] == 'Si']
    return df_filtered

# Funciones para generar cada componente visual dinámicamente según los filtros
def generar_resumen_md(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    total_clubs = len(dff)
    total_juniors = dff['Cantidad de Juniors (menores de 13 años) en la escuela'].sum()
    total_menores = dff['Cantidad de Menores (13 a 18 años) en la escuela'].sum()
    promedio_total_alumnos = dff['Total de alumnos'].mean() if total_clubs > 0 else 0
    # Porcentajes de algunos indicadores
    perc_range = perc_materials = perc_registro = 0
    if total_clubs > 0:
        perc_range = 100 * (dff['Cuenta con una zona de práctica (driving range)?'] == 'Si').sum() / total_clubs
        perc_materials = 100 * (dff['Cuenta con materiales didácticos en la escuela?'] == 'Si').sum() / total_clubs
        if 'Registran información?' in dff.columns:
            perc_registro = 100 * (dff['Registran información?'] == 'Si').sum() / total_clubs
    # Texto en Markdown con las estadísticas principales
    md = f"""**Total de clubes encuestados:** {total_clubs}  
**Total de alumnos:** {int(total_juniors + total_menores)} (Juniors: {int(total_juniors)}, Menores: {int(total_menores)})  
**Promedio de alumnos por club:** {promedio_total_alumnos:.1f}  
**Clubs con Driving Range:** {((dff['Cuenta con una zona de práctica (driving range)?'] == 'Si').sum())} ({perc_range:.0f}% del total)  
**Clubs con materiales didácticos:** {((dff['Cuenta con materiales didácticos en la escuela?'] == 'Si').sum())} ({perc_materials:.0f}% del total)  
**Clubs que registran información de alumnos:** {((dff['Registran información?'] == 'Si').sum() if 'Registran información?' in dff.columns else 0)} ({perc_registro:.0f}% del total)"""
    return pn.pane.Markdown(md, width=400)

def grafico_infraestructura(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    # Contar clubes que poseen cada instalación
    instalaciones = {
        'Driving Range': (dff['Cuenta con una zona de práctica (driving range)?'] == 'Si').sum(),
        'Putting Green': (dff['El club cuenta con putting green?'] == 'Si').sum() if 'El club cuenta con putting green?' in dff.columns else 0,
        'Green de chipping': (dff['Cuenta con un green para chipping?'] == 'Si').sum() if 'Cuenta con un green para chipping?' in dff.columns else 0,
        'Bunker de práctica': (dff['Cuenta con un bunker de práctica?'] == 'Si').sum() if 'Cuenta con un bunker de práctica?' in dff.columns else 0,
        'Cancha junior (Eagles/Birdies)': (dff['Tiene demarcación de campo propio para juniors?  (Eagles y Birdies)'] == 'Si').sum() if 'Tiene demarcación de campo propio para juniors?  (Eagles y Birdies)' in dff.columns else 0
    }
    inst_df = pd.DataFrame({'Instalación': list(instalaciones.keys()), 'Cantidad de clubes': list(instalaciones.values())})
    fig = px.bar(inst_df, x='Instalación', y='Cantidad de clubes', title='Disponibilidad de instalaciones')
    fig.update_layout(yaxis=dict(range=[0, len(dff)]))
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_materiales_bar(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    # Contar cada tipo de material didáctico
    materiales_cols = [c for c in dff.columns if c.startswith('Cuales?/') and c in [
        'Cuales?/Pelotas de practica','Cuales?/Juego de palos','Cuales?/Palos para distintas edades',
        'Cuales?/Palos para zurdos','Cuales?/Conos','Cuales?/Aros','Cuales?/varas','Cuales?/Alfombras','Cuales?/Otros']]
    materiales_nombres = [col.split('/')[-1] for col in materiales_cols]  # obtener nombre después de "Cuales?/"
    conteos = [(dff[col] == 1).sum() for col in materiales_cols]
    mat_df = pd.DataFrame({'Material': materiales_nombres, 'Cantidad de clubes': conteos})
    fig = px.bar(mat_df, x='Material', y='Cantidad de clubes', title='Tipos de materiales didácticos disponibles')
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_materiales_pie(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    # Porcentaje de clubes con materiales vs sin materiales
    if len(dff) == 0:
        fig = px.pie(values=[0], names=['Sin datos'])
    else:
        counts = dff['Cuenta con materiales didácticos en la escuela?'].value_counts()
        labels = counts.index.tolist()  # 'Si'/'No'
        fig = px.pie(values=counts.values, names=labels, title='¿Cuentan con materiales?')
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_aspectos(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    # Sumar evaluación de aspectos (columnas de aspectos evaluados)
    aspectos_cols = [col for col in dff.columns if col.startswith('¿Qué aspectos evalúa')]
    if not aspectos_cols:
        return pn.pane.Markdown("*(No hay datos de aspectos evaluados)*")
    aspecto_nombres = [col.split('/')[-1] for col in aspectos_cols]
    conteos = [(dff[col] == 1).sum() for col in aspectos_cols]
    asp_df = pd.DataFrame({'Aspecto': aspecto_nombres, 'Número de clubes': conteos})
    fig = px.bar(asp_df, x='Número de clubes', y='Aspecto', orientation='h', title='Aspectos evaluados a los alumnos')
    fig.update_layout(xaxis=dict(range=[0, len(dff)]))
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_metodos(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    metodos_cols = [col for col in dff.columns if col.startswith('¿Qué métodos o herramientas utiliza')]
    if not metodos_cols:
        return pn.pane.Markdown("*(No hay datos de métodos de evaluación)*")
    metodo_nombres = [col.split('/')[-1] for col in metodos_cols]
    conteos = [(dff[col] == 1).sum() for col in metodos_cols]
    met_df = pd.DataFrame({'Método': metodo_nombres, 'Número de clubes': conteos})
    fig = px.bar(met_df, x='Número de clubes', y='Método', orientation='h', title='Métodos de evaluación utilizados')
    fig.update_layout(xaxis=dict(range=[0, len(dff)]))
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_registros_pie(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    if 'Registran información?' not in dff.columns or len(dff) == 0:
        fig = px.pie(values=[0], names=['Sin datos'])
    else:
        counts = dff['Registran información?'].value_counts()
        labels = counts.index.tolist()
        fig = px.pie(values=counts.values, names=labels, title='¿Registran información de alumnos?')
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_registros_bar(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    registro_cols = [col for col in dff.columns if col.startswith('¿Qué tipo de información registran?')]
    # Si no hay columnas de registro o si ningún club registra (todos 'No'), no mostrar gráfico
    if not registro_cols or (dff['Registran información?'] == 'No').all():
        return pn.pane.Markdown("*(No hay datos de registros)*")
    registro_nombres = [col.split('/')[-1] for col in registro_cols]
    conteos = [(dff[col] == 1).sum() for col in registro_cols]
    reg_df = pd.DataFrame({'Tipo de registro': registro_nombres, 'Número de clubes': conteos})
    fig = px.bar(reg_df, x='Número de clubes', y='Tipo de registro', orientation='h', title='Tipos de información registrada')
    fig.update_layout(xaxis=dict(range=[0, len(dff)]))
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_formacion_bar(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    # Contar tipos de formación del profesor
    formacion_cols = [col for col in dff.columns if col.startswith('Cuales?/') and col in [
        'Cuales?/Título/Certificación de Instructor o Profesional de Golf',
        'Cuales?/Título en Educación Física o similar',
        'Cuales?/Curso de especialización en enseñanza de golf para menores',
        'Cuales?/Clínicas o talleres informales de golf',
        'Cuales?/Otro']]
    if not formacion_cols:
        return pn.pane.Markdown("*(No hay datos de formación de profesores)*")
    formacion_nombres = [col.split('/')[-1] for col in formacion_cols]
    conteos = [(dff[col] == 1).sum() for col in formacion_cols]
    form_df = pd.DataFrame({'Formación del instructor': formacion_nombres, 'Cantidad de clubes': conteos})
    fig = px.bar(form_df, x='Formación del instructor', y='Cantidad de clubes', title='Formación/certificaciones de los instructores')
    return pn.pane.Plotly(fig, config={'responsive': True})

def grafico_hist_alumnos(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    if len(dff) == 0:
        return pn.pane.Markdown("*(No hay datos para mostrar distribución)*")
    fig = px.histogram(dff, x='Total de alumnos', nbins=5, title='Distribución del total de alumnos por club')
    return pn.pane.Plotly(fig, config={'responsive': True})

def tabla_datos(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    # Excluir columnas técnicas/metadatos para mayor claridad en la tabla
    cols_to_drop = [c for c in dff.columns if c.startswith('_') or c in [
        'start','end','.','Ubicación','Ubicacion','Codigo de club: ${Nombre_del_club}',
        '__version__','_tags','#####<span style="color:green"; "font-family:cursive">_Gracias por su participación_</span>']]
    df_tabla = dff.drop(columns=[c for c in cols_to_drop if c in dff.columns], errors='ignore')
    # Crear tabla interactiva (Tabulator) ordenable y filtrable
    tab = pn.widgets.Tabulator(df_tabla, pagination='local', page_size=10, sizing_mode='stretch_width',
                               frozen_columns=['Nombre del club'], header_filters=True)
    return tab

def mapa_clubes(clubs, has_range, has_materials):
    dff = filtrar_datos(clubs, has_range, has_materials)
    if len(dff) == 0 or dff['latitude'].isna().all():
        return pn.pane.Markdown("*(No hay ubicaciones para mostrar)*")
    # Calcular centro del mapa (promedio de coordenadas)
    lat_center = dff['latitude'].mean()
    lon_center = dff['longitude'].mean()
    # Ajustar zoom: más cerca si solo hay un club seleccionado
    zoom = 5
    if len(dff) == 1:
        zoom = 10
    # Crear mapa con Plotly (OpenStreetMap tiles)
    fig = px.scatter_mapbox(dff, lat='latitude', lon='longitude', hover_name='Nombre del club',
                             hover_data={'Cantidad de Juniors (menores de 13 años) en la escuela': True,
                                         'Cantidad de Menores (13 a 18 años) en la escuela': True},
                             size='Total de alumnos', size_max=30, zoom=zoom,
                             center={'lat': lat_center, 'lon': lon_center}, mapbox_style='open-street-map')
    fig.update_layout(title='Mapa de clubes', margin={"r":0, "t":50, "l":0, "b":0})
    return pn.pane.Plotly(fig, config={'responsive': True})

# Crear componentes reactivamente vinculados a los filtros
resumen_panel = pn.bind(generar_resumen_md, club_filter, has_range_filter, has_materials_filter)
infra_bar_panel = pn.bind(grafico_infraestructura, club_filter, has_range_filter, has_materials_filter)
materials_pie_panel = pn.bind(grafico_materiales_pie, club_filter, has_range_filter, has_materials_filter)
materials_bar_panel = pn.bind(grafico_materiales_bar, club_filter, has_range_filter, has_materials_filter)
aspects_bar_panel = pn.bind(grafico_aspectos, club_filter, has_range_filter, has_materials_filter)
methods_bar_panel = pn.bind(grafico_metodos, club_filter, has_range_filter, has_materials_filter)
registros_pie_panel = pn.bind(grafico_registros_pie, club_filter, has_range_filter, has_materials_filter)
registros_bar_panel = pn.bind(grafico_registros_bar, club_filter, has_range_filter, has_materials_filter)
formacion_bar_panel = pn.bind(grafico_formacion_bar, club_filter, has_range_filter, has_materials_filter)
hist_panel = pn.bind(grafico_hist_alumnos, club_filter, has_range_filter, has_materials_filter)
tabla_panel = pn.bind(tabla_datos, club_filter, has_range_filter, has_materials_filter)
mapa_panel = pn.bind(mapa_clubes, club_filter, has_range_filter, has_materials_filter)

# Organizar los componentes en pestañas (Tabs)
tabs = pn.Tabs(
    ('Resumen', pn.Column(resumen_panel, hist_panel)),
    ('Infraestructura', infra_bar_panel),
    ('Materiales', pn.Column(pn.Row(materials_pie_panel, materials_bar_panel))),
    ('Evaluación', pn.Column(aspects_bar_panel, methods_bar_panel)),
    ('Registros', pn.Column(pn.Row(registros_pie_panel, registros_bar_panel))),
    ('Formación', formacion_bar_panel),
    ('Datos', tabla_panel),
    ('Mapa', mapa_panel)
)

# Construir la plantilla con encabezado, menú lateral y las pestañas
template = pn.template.FastListTemplate(
    title='Relevamiento de Escuelas de Golf',
    sidebar=[pn.pane.Markdown('## Filtros'), club_filter, has_range_filter, has_materials_filter],
    main=[tabs]
)

# Exportar el panel a un archivo HTML estático
template.save('panel_escuelas_golf.html', resources=INLINE)
print("Panel guardado como 'panel_escuelas_golf.html'")


Panel guardado como 'panel_escuelas_golf.html'
